In [1]:
import pandas as pd 
import sqlite3
import polars as pl
import duckdb
import time

# Pandas and SQLite

In [2]:
# students_df = pd.read_csv("data/students.csv")
# grades_df = pd.read_csv("data/grades.csv")
# attendance_df = pd.read_csv("data/attendance.csv")

# conn = sqlite3.connect("school_data_lite.db")

# #  df to the db as tables
# students_df.to_sql("students", conn, if_exists="replace", index=False)
# grades_df.to_sql("grades", conn, if_exists="replace", index=False)
# attendance_df.to_sql("attendance", conn, if_exists="replace", index=False)

In [15]:
%%timeit -r 20 -n 1
conn = sqlite3.connect("school_data_lite.db")
# Query test
query = """
SELECT 
    students.First_Name, 
    students.Last_Name, 
    attendance.Week_1, 
    AVG(
        grades.Assignment_1 + grades.Assignment_2 + grades.Assignment_3 + grades.Assignment_4 + grades.Assignment_5 +
        grades.Assignment_6 + grades.Assignment_7 + grades.Assignment_8 + grades.Assignment_9 + grades.Assignment_10
    ) / 10.0 AS avg_grade
FROM students
JOIN attendance ON students.Student_ID = attendance.Student_ID
JOIN grades ON students.Student_ID = grades.Student_ID
GROUP BY students.Student_ID, students.First_Name, students.Last_Name, attendance.Week_1
ORDER BY students.First_Name ASC, students.Last_Name ASC;
"""
result = pd.read_sql_query(query, conn)

print(result)

conn.close()

        First_Name Last_Name   Week_1  avg_grade
0        Alexander     Allen  Present       67.2
1        Alexander     Allen   Absent       77.5
2        Alexander     Allen  Present       86.4
3        Alexander     Allen   Absent       79.3
4        Alexander     Allen   Absent       77.4
...            ...       ...      ...        ...
1999995    William     Young  Present       72.2
1999996    William     Young   Absent       76.7
1999997    William     Young   Absent       75.8
1999998    William     Young  Present       78.2
1999999    William     Young   Absent       74.1

[2000000 rows x 4 columns]
        First_Name Last_Name   Week_1  avg_grade
0        Alexander     Allen  Present       67.2
1        Alexander     Allen   Absent       77.5
2        Alexander     Allen  Present       86.4
3        Alexander     Allen   Absent       79.3
4        Alexander     Allen   Absent       77.4
...            ...       ...      ...        ...
1999995    William     Young  Present    

# Polars and DuckDB

In [4]:
# students_df = pl.read_csv("data/students.csv")
# grades_df = pl.read_csv("data/grades.csv")
# attendance_df = pl.read_csv("data/attendance.csv")

# conn = duckdb.connect("school_data.duckdb")

# # make the db
# conn.register("students", students_df)
# conn.register("grades", grades_df)
# conn.register("attendance", attendance_df)

In [16]:
%%timeit -r 20 -n 1
conn = duckdb.connect("school_data.duckdb")

# Example Query: Get student names, attendance for Week 1, and average grade for all assignments
query = """
SELECT 
    students.First_Name, 
    students.Last_Name, 
    attendance.Week_1, 
    AVG(
        grades.Assignment_1 + grades.Assignment_2 + grades.Assignment_3 + grades.Assignment_4 + grades.Assignment_5 +
        grades.Assignment_6 + grades.Assignment_7 + grades.Assignment_8 + grades.Assignment_9 + grades.Assignment_10
    ) / 10.0 AS avg_grade
FROM students
JOIN attendance ON students.Student_ID = attendance.Student_ID
JOIN grades ON students.Student_ID = grades.Student_ID
GROUP BY students.Student_ID, students.First_Name, students.Last_Name, attendance.Week_1
ORDER BY students.First_Name ASC, students.Last_Name ASC;
"""


# Execute the query
result = conn.execute(query).fetchdf()

# Step 5: Display the result
print(result)

# Close the connection
conn.close()

        First_Name Last_Name   Week_1  avg_grade
0        Alexander     Allen   Absent       71.9
1        Alexander     Allen  Present       84.3
2        Alexander     Allen  Present       74.3
3        Alexander     Allen  Present       74.4
4        Alexander     Allen   Absent       75.1
...            ...       ...      ...        ...
1999995    William     Young  Present       74.4
1999996    William     Young  Present       77.6
1999997    William     Young   Absent       64.6
1999998    William     Young   Absent       68.5
1999999    William     Young   Absent       75.4

[2000000 rows x 4 columns]
        First_Name Last_Name   Week_1  avg_grade
0        Alexander     Allen   Absent       78.9
1        Alexander     Allen  Present       63.8
2        Alexander     Allen  Present       79.6
3        Alexander     Allen   Absent       80.9
4        Alexander     Allen   Absent       78.7
...            ...       ...      ...        ...
1999995    William     Young  Present    

In [17]:
conn.close()

# results

- Polars: 346 milliseconds ± 11.5 milliseconds per loop
- Pandas: 6.78 seconds ± 166 milliseconds per loop

Polars is significantly faster than Pandas in this case, with a runtime of around 346 ms, compared to 6.78 seconds for Pandas.

**Polars is approximately 19.6 times faster**

The database contains 25 cols with 2 million rows inside of 3 tables. 

polars and Duck DB will be the backend. 